In [1]:
import numpy as np 
import cv2 
import tensorflow as tf
import keras
import os 
from random import shuffle 
import matplotlib.pyplot as plt
from keras.models import load_model
from tqdm import tqdm
import pickle
import tensorflow as tf
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
MODEL_NAME = 'model.tflearn'
IMG_SIZE = 32
#LR = 1e-3
LR = 0.00146

In [3]:
train_data = np.load('train_data.npy') 
test_data = np.load('test_data.npy') 

In [4]:
tf.reset_default_graph() 
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE,3], name ='input')   
#conv_2d(convnet, weight, SIZE OF MASK, activation ='relu')

convnet = conv_2d(convnet, 32, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3) 
#convnet = dropout(convnet, 0.3)

convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  
convnet = conv_2d(convnet, 128, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  
convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  

convnet = fully_connected(convnet, 1024, activation ='relu') 
convnet = dropout(convnet, 0.3)
  
convnet = fully_connected(convnet, 5, activation ='softmax')

convnet = regression(convnet, optimizer ='adam', learning_rate = LR, 
      loss ='categorical_crossentropy', name ='targets') 
  
model = tflearn.DNN(convnet) 
  

model.load(MODEL_NAME)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\John-jackson\model.tflearn


In [27]:
ans1 = []
#IMG_SIZE = 5000
#model.load('./roadsignslassification.hdf5')

#***NO HONK,SCHOOL AHEAD,NO ENTRY,STOP well trained***#

#img_data = r'E:/work/dataset/predict/no_honk/no_honk_predict4.jpg'
#img_data = r'E:/work/dataset/predict/no_honk/1.jpg'
#img_data = r'E:/work/dataset/predict/no_u_turn/no_u_turn_predict3.jpg'
#img_data = r'E:/dataset/predict/school_ahead/school-ahead-predict(3).jpg'
img_data = r'E:/dataset/predict/no_entry/no_entry_predict1.jpg'
#img_data = r'E:/work/dataset/predict/stop/1 (10).jpg'
#img_data = r'E:/work/dataset/predict/test.jpg'
#OR Enter an image here

im = cv2.imread(img_data)

im = cv2.resize(im, (5000, 5000))

hsv_img = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
COLOR_MIN = np.array([169, 100, 100],np.uint8)
COLOR_MAX = np.array([189, 255, 255],np.uint8)
frame_threshed = cv2.inRange(hsv_img, COLOR_MIN, COLOR_MAX)

imgray = frame_threshed
ret,thresh = cv2.threshold(frame_threshed,127,255,0)
_,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# Find the index of the largest contour
areas = [cv2.contourArea(c) for c in contours]
max_index = np.argmax(areas)

cnt=contours[max_index]
#make the bounding rectangle
x,y,w,h = cv2.boundingRect(cnt)
cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)

#crop the rectangle
crop_img = im[y:y+h, x:x+w] #this is a rgb image
cv2.imwrite('sign.jpg', crop_img) #this is a rgb image

cv2.imshow("cropped",crop_img)
test = cv2.resize(crop_img,(IMG_SIZE,IMG_SIZE))
model_out = model.predict([test])

index = np.argmax(model_out)

if index == 0: 
    ans1.append('no_honking')
elif index == 1:
    ans1.append('school_ahead')
elif index == 2:
    ans1.append('stop')
elif index == 3:
    ans1.append('no_u_turn')
elif index == 4:
    ans1.append('no_entry')
else:
    ans1.append('Cannot classify')

cv2.waitKey(0)
cv2.destroyAllWindows()
plt.imshow(im)
plt.show()
print(ans1)

ValueError: attempt to get argmax of an empty sequence